# Linear Regression

### Import Libraries

#### Linear Model

Equation:
$y=\beta_0+\beta_1x_1+...+\beta_nx_n+\epsilon$
Where:
- y is the response variable
- $\beta_0$ is the intercept (**model.intercept_**)
- $\beta_i$ is the coefficient of the i feature (**model.coef_**)
- $x_i$ is the i_th feature

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from mpl_toolkits.mplot3d import axes3d

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

# Visualization Style
plt.style.use('seaborn-white')
#plt.style.use('fivethirtyeight')
plt.tight_layout()


### Import Datasets

In [ ]:
advertising = pd.read_csv('../datasets/Advertising.csv', usecols=[1,2,3,4])
advertising.info()

### Exploratory Data Analysis

In [ ]:
advertising.shape

In [ ]:
advertising.head()

In [ ]:
advertising.info()

In [ ]:
advertising.describe()

### Data Visualization

#### Ouliers Visualization

In [ ]:
fig, axes = plt.subplots(4, figsize = (5,5))
sns.boxplot(advertising['TV'], ax = axes[0])
sns.boxplot(advertising['radio'], ax = axes[1])
sns.boxplot(advertising['newspaper'], ax = axes[2])
sns.boxplot(advertising['sales'], ax = axes[3])
plt.show()

#### Relation Visualization: Sales vs Others

In [ ]:
sns.pairplot(advertising, x_vars=['TV', 'newspaper', 'radio'], 
             y_vars='sales', height=4, aspect=1, kind='scatter')
plt.show()

#### Correlation

In [ ]:
sns.heatmap(advertising.corr(), cmap="YlGnBu", annot = True)
plt.show()

### Simple Linear Regression: Sales vs TV Advertising

#### Model Creation

In [ ]:
regr = LinearRegression()

#### Prepare the Data

In [ ]:
X = advertising['TV'].values.reshape(-1,1)
y = advertising['sales']

#### Train-Test Data Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

#### Train the Model

In [ ]:
regr.fit(X_train, y_train)

#### Get the Coefficients

In [ ]:
print("Model slope:", regr.coef_[0])
print("Model intercept:", regr.intercept_)

#### Visualize the Fit on the Training Set

In [ ]:
plt.scatter(X_train, y_train)
plt.plot(X_train, regr.intercept_ + regr.coef_[0] * X_train, 'r')
plt.show()

### Model Evaluation

#### Predictions on the Test Set

In [ ]:
y_pred = regr.predict(X_test)

#### Calculate MSE (Mean Squared Error)

In [ ]:
np.sqrt(mean_squared_error(y_test, y_pred))

#### Visualize the Fit on the Test Set

In [ ]:
plt.scatter(X_test, y_test)
plt.plot(X_test, regr.intercept_ + regr.coef_[0] * X_test, 'r')
plt.show()

#### Errors Analysis

#### Calculate Errors on Training Set

In [ ]:
y_train_pred = regr.predict(X_train)
res = (y_train - y_train_pred)

#### Visualize Errors Distribution

In [ ]:
sns.distplot(res, bins = 15)
plt.title('Errors Distribution')
plt.show()

### Regression Coefficients

In [ ]:
type(pd.DataFrame(advertising.TV))

In [ ]:
# Regression coefficients (Ordinary Least Squares)
regr = LinearRegression()

X = scale(advertising.TV, with_mean=True, with_std=False).reshape(-1,1)
y = advertising.sales

regr.fit(X,y)
print(regr.intercept_)
print(regr.coef_)

In [ ]:
# Create grid coordinates for plotting
B0 = np.linspace(regr.intercept_-2, regr.intercept_+2, 50)
B1 = np.linspace(regr.coef_-0.02, regr.coef_+0.02, 50)
xx, yy = np.meshgrid(B0, B1, indexing='xy')
Z = np.zeros((B0.size,B1.size))

# Calculate Z-values (RSS) based on grid of coefficients
for (i,j),v in np.ndenumerate(Z):
    Z[i,j] =((y - (xx[i,j]+X.ravel()*yy[i,j]))**2).sum()/1000

# Minimized RSS
min_RSS = r'$\beta_0$, $\beta_1$ for minimized RSS'
min_rss = np.sum((regr.intercept_+regr.coef_*X - y.values.reshape(-1,1))**2)/1000
min_rss

In [ ]:
fig = plt.figure(figsize=(15,6))
fig.suptitle('RSS - Regression coefficients', fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, projection='3d')

# Left plot
CS = ax1.contour(xx, yy, Z, cmap=plt.cm.Set1, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax1.scatter(regr.intercept_, regr.coef_[0], c='r', label=min_RSS)
ax1.clabel(CS, inline=True, fontsize=10, fmt='%1.1f')

# Right plot
ax2.plot_surface(xx, yy, Z, rstride=3, cstride=3, alpha=0.3)
ax2.contour(xx, yy, Z, zdir='z', offset=Z.min(), cmap=plt.cm.Set1,
            alpha=0.4, levels=[2.15, 2.2, 2.3, 2.5, 3])
ax2.scatter3D(regr.intercept_, regr.coef_[0], min_rss, c='r', label=min_RSS)
ax2.set_zlabel('RSS')
ax2.set_zlim(Z.min(),Z.max())
ax2.set_ylim(0.02,0.07)

# settings common to both plots
for ax in fig.axes:
    ax.set_xlabel(r'$\beta_0$', fontsize=17)
    ax.set_ylabel(r'$\beta_1$', fontsize=17)
    ax.set_yticks([0.03,0.04,0.05,0.06])
    ax.legend()

### Multiple Linear Regression: Sales vs Radio + TV Advertising

#### Correlation Matrix

In [ ]:
advertising.corr()

In [ ]:
regr = LinearRegression()

X = advertising[['radio', 'TV']].values
y = advertising.sales

regr.fit(X,y)
print(regr.coef_)
print(regr.intercept_)

In [ ]:
# What are the min/max values of Radio & TV?
# Use these values to set up the grid for plotting.
advertising[['radio', 'TV']].describe()

In [ ]:
# Create a coordinate grid
Radio = np.arange(0,50)
TV = np.arange(0,300)

B1, B2 = np.meshgrid(Radio, TV, indexing='xy')
Z = np.zeros((TV.size, Radio.size))

for (i,j),v in np.ndenumerate(Z):
        Z[i,j] =(regr.intercept_ + B1[i,j]*regr.coef_[0] + B2[i,j]*regr.coef_[1])

In [ ]:
# Create plot
fig = plt.figure(figsize=(10,6))
fig.suptitle('Regression: Sales ~ Radio + TV Advertising', fontsize=20)

ax = axes3d.Axes3D(fig)

ax.plot_surface(B1, B2, Z, rstride=10, cstride=5, alpha=0.4)
ax.scatter3D(advertising.radio, advertising.TV, advertising.sales, c='r')

ax.set_xlabel('Radio')
ax.set_xlim(0,50)
ax.set_ylabel('TV')
ax.set_ylim(bottom=0)
ax.set_zlabel('Sales');